### Import necessary libraries

In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
sys.path.append(r'../func')
import combine_obj
import export_obj

### Load necessary data from csv as dataframe

In [2]:
movie_df = pd.read_csv('..\Harry Potter-20220610T024151Z-001\Movies.csv', sep=',', quotechar='"')
movie_df.head(2)

,Movie ID,Movie Title,Release Year,Runtime,Budget,Box Office
0,1,Harry Potter and the Philosopher's Stone,2001,152,"$125,000,000","$1,002,000,000"
1,2,Harry Potter and the Chamber of Secrets,2002,161,"$100,000,000","$880,300,000"


In [3]:
chapter_df = pd.read_csv('..\Harry Potter-20220610T024151Z-001\Chapters.csv', sep=',', quotechar='"', encoding='Latin-1')
chapter_df.head(2)

,Chapter ID,Chapter Name,Movie ID,Movie Chapter
0,1,Doorstep Delivery,1,1
1,2,The Vanishing Glass,1,2


In [4]:
dialogue_df = pd.read_csv('..\Harry Potter-20220610T024151Z-001\Dialogue.csv', sep=',', quotechar='"', encoding='Latin-1')
dialogue_df.head(2)

,Dialogue ID,Chapter ID,Place ID,Character ID,Dialogue
0,1,1,8,4,I should have known that you would be here...P...
1,2,1,8,7,"Good evening, Professor Dumbledore. Are the ru..."


In [5]:
character_df = pd.read_csv('..\Harry Potter-20220610T024151Z-001\Characters.csv', sep=',', quotechar='"', encoding='Latin-1')
character_df.head(2)

,Character ID,Character Name,Species,Gender,House,Patronus,Wand (Wood),Wand (Core)
0,1,Harry Potter,Human,Male,Gryffindor,Stag,Holly,Phoenix Feather
1,2,Ron Weasley,Human,Male,Gryffindor,Jack Russell Terrier,NaN,NaN


### Analyze

#### Generate basic data

In [6]:
dialogue_chapter_df = dialogue_df.merge(chapter_df, how='left' ,left_on='Chapter ID', right_on='Chapter ID')
dialogue_chapter_df.head(2)

,Dialogue ID,Chapter ID,Place ID,Character ID,Dialogue,Chapter Name,Movie ID,Movie Chapter
0,1,1,8,4,I should have known that you would be here...P...,Doorstep Delivery,1,1
1,2,1,8,7,"Good evening, Professor Dumbledore. Are the ru...",Doorstep Delivery,1,1


#### Dialogue count by character across movies

In [7]:
groupBy_character_df = dialogue_chapter_df.groupby(['Character ID'])['Character ID'].count().reset_index(name='Dialogue count')
groupBy_character_df = groupBy_character_df.merge(character_df, how='left' ,left_on='Character ID', right_on='Character ID')
groupBy_character_df.head(2)

,Character ID,Dialogue count,Character Name,Species,Gender,House,Patronus,Wand (Wood),Wand (Core)
0,1,1922,Harry Potter,Human,Male,Gryffindor,Stag,Holly,Phoenix Feather
1,2,865,Ron Weasley,Human,Male,Gryffindor,Jack Russell Terrier,NaN,NaN


In [8]:
# prepare chart data
chart_title = 'No. of dialogue by character across movies'
x_label = 'Character'
x = groupBy_character_df['Character Name']
y1_label = 'Dialogue count'
y1 = groupBy_character_df['Dialogue count']
# Create figure with secondary y-axis
fig = make_subplots()
# Add traces
fig.add_trace(
    go.Bar(x=x, y=y1, name=y1_label, width=.35),
)
# Add figure title
fig.update_layout(
    title_text=chart_title,
    font=dict(size=14),
    xaxis={'categoryorder':'total descending'},
)
fig.update_xaxes(dtick=5)
fig.show()

In [9]:
fig = make_subplots()

fig.add_trace(
    go.Bar(x=y1, y=x,name=x_label, width=.7, orientation='h')
)

fig.update_layout(
    title_text=chart_title,
    height=600,
    barmode='stack',
    font=dict(size=14),
    bargap=0.5,
    yaxis={'categoryorder':'total descending'},
)

fig.show()

In [10]:
char_id = 'fe10a966-2650-4eb2-a5af-6b3151c5cff9'
export_obj.export_chart_to_html(
    fig=fig, 
    height=500, 
    chart_id=char_id,
    chart_title=chart_title,
    path_to_filename='../docs/{}.html'.format(char_id), 
    describtion_list=[
        {'This chart displays the total number of dialogues by character in the franchise:': ['The x-axis will be the number', 'The y-axis will be the character name']}
    ], 
    insight_list=[
        'Harry Potter has the highest number of dialogues which is 1922 => answering for the question 2',
        'The lowest number of dialogues is 2, which does belong to several characters like Bloody Baron, Alecto Carrow',
        'The postion of the 1st rank is much important than the 2nd and the 3rd positions. The 2nd position (Ron Weasly - 865 dialogues) and the 3rd position (Hermione Granger - 848 dialogues) have quite similar count, but the count of the 1st position is more 2 times than the next 2 positions'
    ]
)

#### Dialogue count by each character per movie

In [11]:
groupBy_movie_df = dialogue_chapter_df.groupby(['Movie ID', 'Character ID'])['Movie ID'].count().reset_index(name='Dialogue count')
groupBy_movie_df.head(2)

,Movie ID,Character ID,Dialogue count
0,1,1,242
1,1,2,142


In [12]:
idx = groupBy_movie_df.groupby(['Movie ID'])['Dialogue count'].transform(max) == groupBy_movie_df['Dialogue count']
max_df = groupBy_movie_df[idx]
max_df = max_df.merge(movie_df, how='left' ,left_on='Movie ID', right_on='Movie ID')
max_df = max_df.merge(character_df, how='left' ,left_on='Character ID', right_on='Character ID')
max_df

,Movie ID,Character ID,Dialogue count,Movie Title,Release Year,Runtime,Budget,Box Office,Character Name,Species,Gender,House,Patronus,Wand (Wood),Wand (Core)
0,1,1,242,Harry Potter and the Philosopher's Stone,2001,152,"$125,000,000","$1,002,000,000",Harry Potter,Human,Male,Gryffindor,Stag,Holly,Phoenix Feather
1,2,1,268,Harry Potter and the Chamber of Secrets,2002,161,"$100,000,000","$880,300,000",Harry Potter,Human,Male,Gryffindor,Stag,Holly,Phoenix Feather
2,3,1,203,Harry Potter and the Prisoner of Azkaban,2004,142,"$130,000,000","$796,700,000",Harry Potter,Human,Male,Gryffindor,Stag,Holly,Phoenix Feather
3,4,1,161,Harry Potter and the Goblet of Fire,2005,157,"$150,000,000","$896,400,000",Harry Potter,Human,Male,Gryffindor,Stag,Holly,Phoenix Feather
4,5,1,314,Harry Potter and the Order of the Phoenix,2007,138,"$150,000,000","$942,000,000",Harry Potter,Human,Male,Gryffindor,Stag,Holly,Phoenix Feather
5,6,1,266,Harry Potter and the Half-Blood Prince,2009,153,"$250,000,000","$943,200,000",Harry Potter,Human,Male,Gryffindor,Stag,Holly,Phoenix Feather
6,7,1,282,Harry Potter and the Deathly Hallows Part 1,2010,146,"$200,000,000","$976,900,000",Harry Potter,Human,Male,Gryffindor,Stag,Holly,Phoenix Feather
7,8,1,186,Harry Potter and the Deathly Hallows Part 2,2011,130,"$250,000,000","$1,342,000,000",Harry Potter,Human,Male,Gryffindor,Stag,Holly,Phoenix Feather


In [13]:
# prepare chart data
chart_title = 'No. of dialogue of {} per movie'.format(max_df['Character Name'][0])
x_label = 'Movie'
x = combine_obj.combine_2_series(max_df['Movie Title'], max_df['Release Year'])
annotation_text = max_df['Character Name']
y1_label = 'Dialogue count'
y1 = max_df['Dialogue count']
# Create figure with secondary y-axis
fig = make_subplots()
# Add traces
fig.add_trace(
    go.Bar(x=y1, y=x,name=x_label, width=.7, orientation='h', text=annotation_text, insidetextanchor="start")
)
# Add figure title
fig.update_layout(
    title_text=chart_title,
    barmode='stack',
    font=dict(size=14),
    bargap=0.5
)

fig.show()

In [14]:
char_id = '34d98515-2207-49ee-a3a0-4099330873be'
export_obj.export_chart_to_html(
    fig=fig, 
    height=600, 
    chart_id=char_id,
    chart_title=chart_title,
    path_to_filename='../docs/{}.html'.format(char_id), 
    describtion_list=[
        {'This chart displays the total number of dialogues of the character having the highest number of dialogues of each movie': [
            'The x-axis will be the number of dialogue', 
            'The y-axis will be the movie name',
            'The name is inside of the bar will be the character name'
        ]},
        'The year is adjacent to the movie name is the release year'
    ], 
    insight_list=[
        'Harry Potter has the highest number of dialogues in every movies'
    ]
)